
Change runtime type to T4 gpu

In [ ]:
# Step 1: Set up your Colab environment
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install necessary libraries
!pip install imageio imageio-ffmpeg numpy torch ffmpeg-python
!pip install opencv-python opencv-python-headless

In [ ]:
# Step 2: Download and prepare the first-order-model
import os
os.chdir('/content/drive/MyDrive')

# Clone the first-order-model repository
!git clone https://github.com/AliaksandrSiarohin/first-order-model.git
os.chdir('first-order-model')

# Download the pre-trained checkpoint
import gdown

# Define the URL and output directory for the checkpoint
url = 'https://drive.google.com/uc?id=1L8P-hpBhZi8Q_1vP2KlQ4N6dvlzpYBvZ'
checkpoint_path = 'vox-adv-cpk.pth.tar'

# Download the checkpoint
gdown.download(url, checkpoint_path, quiet=False)

# https://drive.google.com/file/d/1BDMiyG1j4WWqupSpLzXG7Qmen38gGxGb/view?usp=sharing
# https://drive.google.com/uc?id=1eeWryeq4pYjNTdnWTyUSPBEW8xI8g2bb&export=download

# # Download the audio
# audio_url = 'https://drive.google.com/uc?id=1BDMiyG1j4WWqupSpLzXG7Qmen38gGxGb&export=download'
# audio_path = 's1.mp3'
# gdown.download(audio_url, audio_path, quiet=False)

Cloning into 'first-order-model'...
remote: Enumerating objects: 397, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 397 (delta 38), reused 37 (delta 36), pack-reused 345 (from 3)
Receiving objects: 100% (397/397), 72.19 MiB | 19.88 MiB/s, done.
Resolving deltas: 100% (205/205), done.
Updating files: 100% (48/48), done.


Downloading...
From (original): https://drive.google.com/uc?id=1L8P-hpBhZi8Q_1vP2KlQ4N6dvlzpYBvZ
From (redirected): https://drive.google.com/uc?id=1L8P-hpBhZi8Q_1vP2KlQ4N6dvlzpYBvZ&confirm=t&uuid=b197a1f7-edbc-4f8b-bbf4-a764ce3d248d
To: /content/drive/MyDrive/first-order-model/vox-adv-cpk.pth.tar
100%|██████████| 751M/751M [00:11<00:00, 64.6MB/s]


'vox-adv-cpk.pth.tar'

In [ ]:
import imageio
from moviepy.editor import ImageSequenceClip
import numpy as np
from skimage.transform import resize

def process_frame(frame, crop_coords, target_shape):
    x1, y1, x2, y2 = crop_coords
    cropped_frame = frame[y1:y2, x1:x2]
    resized_frame = resize(cropped_frame, target_shape, preserve_range=True)

    return resized_frame.astype(np.uint8)

def gif_to_video(gif_path, output_video_path, crop_coords, target_shape, fps=30):
    # Load GIF
    gif = imageio.mimread(gif_path)

    # Process frames
    processed_frames = [process_frame(frame, crop_coords, target_shape) for frame in gif]

    processed_frames = processed_frames

    print(len(processed_frames))

    # Save frames as MP4
    clip = ImageSequenceClip(processed_frames, fps=fps)
    clip.write_videofile(output_video_path, codec='libx264')

# Example usage
gif_path = 'sup-mat/vox-teaser.gif'
output_video_path = 'sample_driving_video.mp4'
crop_coords = (60, 60, 450, 450)  # Crop coordinates (x1, y1, x2, y2)
target_shape = (256, 256)  # Target shape (height, width)

gif_to_video(gif_path, output_video_path, crop_coords, target_shape)

  if event.key is 'enter':



43
Moviepy - Building video sample_driving_video.mp4.
Moviepy - Writing video sample_driving_video.mp4



Moviepy - Done !
Moviepy - video ready sample_driving_video.mp4


# Upload Your Image with clear Face

In [ ]:

# Step 3: Upload your image
from google.colab import files

# Allow user to upload an image
uploaded = files.upload()

# Get the filename of the uploaded image
image_path = next(iter(uploaded))


StopIteration: 

In [ ]:
import cv2
import imageio
from google.colab.patches import cv2_imshow
from skimage.transform import resize
from demo import load_checkpoints, make_animation
from skimage import img_as_ubyte
import numpy as np
import torch, ffmpeg
from moviepy.editor import VideoFileClip, AudioFileClip

def detect_crop_faces_and_animate(image_path, driving_video_path='sample_driving_video.mp4', output_file='output.mp4', audio_path='s1.mp3'):
    # Load the Haar Cascade classifier for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Read the image
    image = cv2.imread(image_path)
    image = cv2.resize(image, (256, 256))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=3, minSize=(10, 10))

    if len(faces) == 0:
        print("No faces detected.")
        return

    print(f"Found {len(faces)} faces.")

    animated_images = []

    for i, (x, y, w, h) in enumerate(faces):
        print(f'Processing face {i+1} animation... Please wait...')

        # Crop the detected face with a margin
        margin = 20
        if(len(faces) > 1):
            margin = 10

        x1, y1, x2, y2 = max(0, x - margin), max(0, y - margin), min(image.shape[1], x + w + margin), min(image.shape[0], y + h + margin)
        face = image[y1:y2, x1:x2]

        # Convert cropped face to RGB format
        face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)

        # Load pre-trained checkpoints
        generator, kp_detector = load_checkpoints(config_path='config/vox-adv-256.yaml', checkpoint_path='vox-adv-cpk.pth.tar')

        # Resize source image to match the model requirements
        source_image = resize(face_rgb, (256, 256))[..., :3]

        # Load driving video
        driving_video = imageio.mimread(driving_video_path)

        # Resize driving video frames to match the model requirements
        driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

        # Perform the animation
        predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

        # Convert predictions to uint8 for blending with original image
        predictions = [cv2.cvtColor(img_as_ubyte(pred),cv2.COLOR_BGR2RGB) for pred in predictions]

        # Overlay animated face on the original image
        for j,frame in enumerate(predictions):
            # Resize frame to match the size of the cropped face
            frame_resized = cv2.resize(frame, (x2 - x1, y2 - y1))

            if(i==0):
                animated_image = np.copy(image)
                animated_image[y1:y2, x1:x2] = frame_resized
                animated_images.append(animated_image)
            else:
                animated_images[j][y1:y2, x1:x2] = frame_resized

    animated_images = animated_images * 20

    # Save the animated frames as a video
    fps = 5
    height, width, _ = image.shape
    temp_output = 'temp_output.mp4'
    output_video = cv2.VideoWriter(temp_output, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    for animated_frame in animated_images:
        output_video.write(animated_frame)

    output_video.release()

    # Merge video and audio using ffmpeg-python
    video = ffmpeg.input(temp_output)
    audio = ffmpeg.input(audio_path)
    ffmpeg.output(audio, video, output_file, vcodec='copy', acodec='copy').run()

    print(f"\n\nAnimation with custom audio saved as {output_file}")


In [ ]:
# Example usage
import random
rand_int = random.randint(0, 10000)

# Birthday special
# audio_url = 'https://drive.google.com/uc?id=1BDMiyG1j4WWqupSpLzXG7Qmen38gGxGb&export=download'
# audio_path = 's1.mp3'
# gdown.download(audio_url, audio_path, quiet=False)

# Ncc speech - https://drive.google.com/file/d/1bEqIhlcpr0eGLn_dG0x8fU6ClbWJ1hAU/view?usp=sharing
audio_url = 'https://drive.google.com/uc?id=1bEqIhlcpr0eGLn_dG0x8fU6ClbWJ1hAU&export=download'
audio_path = 'NCC.mp3'
gdown.download(audio_url, audio_path, quiet=False)

# if u have upload, then comment below
# image_url = https://drive.google.com/file/d/1EMFy0zkqH-qUsGx0fDHNK3YeD8cLykix/view?usp=sharing
image_url = 'https://drive.google.com/uc?id=1EMFy0zkqH-qUsGx0fDHNK3YeD8cLykix&export=download'
image_path = 'test.jpeg'
gdown.download(image_url, image_path, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1bEqIhlcpr0eGLn_dG0x8fU6ClbWJ1hAU&export=download
To: /content/drive/MyDrive/first-order-model/NCC.mp3
100%|██████████| 737k/737k [00:00<00:00, 75.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1EMFy0zkqH-qUsGx0fDHNK3YeD8cLykix&export=download
To: /content/drive/MyDrive/first-order-model/test.jpeg
100%|██████████| 1.33M/1.33M [00:00<00:00, 110MB/s]


'test.jpeg'

In [ ]:


# image_path = '/content/Purushotham.jpeg'
output_file = f'animate_{rand_int}.mp4'
driving_video_path = 'sample_driving_video.mp4'
audio_path = 'NCC.mp3'

detect_crop_faces_and_animate(image_path,  driving_video_path, output_file,audio_path)

Found 1 faces.
Processing face 1 animation... Please wait...


  checkpoint = torch.load(checkpoint_path)



  0%|          | 0/44 [00:00<?, ?it/s]

  warnings.warn(





Animation with custom audio saved as animate_9000.mp4


In [ ]:
from google.colab import files

# Download the file
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Text To Speech
https://ttsmaker.com/

Good morning, respected instructors, fellow cadets, and friends.

As we step into the new year, it’s a perfect time to reflect on the lessons we’ve learned and to set new goals. The NCC has taught us the value of discipline, leadership, and resilience. It has shown us that growth comes not from avoiding challenges, but from facing them head-on and pushing beyond our limits.

This year, let’s embrace change with open arms. Change is inevitable, but it is also the path to progress. Set new goals—whether it’s improving your fitness, learning new skills, or stepping up as a leader. Remember, progress is made one small step at a time.

Discipline is the foundation of success. This year, let’s commit to building strong habits—waking up early, staying focused, and giving our best in everything we do. Success doesn’t come overnight; it comes through consistent effort.

Let’s also remember the importance of leadership. True leadership is not about authority; it’s about inspiring others, serving with integrity, and leading by example. As cadets, we are not just training for ourselves; we’re preparing to serve our nation and our community.

Lastly, let’s be resilient. Challenges will come, but we must rise every time we fall. The year ahead will bring obstacles, but with perseverance, we will overcome them and emerge stronger.

So, let’s make this year count. Let’s work hard, stay disciplined, and lead with courage. Together, we will make 2024 our best year yet.

Happy New Year, everyone! Let’s make it happen.

In [ ]:
# Text to Speech